In [2]:
import random

In [3]:
# Constants
POPULATION_SIZE = 5
GENOME_LENGTH = 8  # 8 bits to represent values from -15 to 15
GENERATIONS = 5
MUTATION_RATE = 0.01
CROSSOVER_RATE = 0.7
TOURNAMENT_SIZE = 2  # Can be adjusted
ELITISM_COUNT = 1  # Number of elites to carry forward

In [4]:
# Decode binary string into an integer in the range [-15, 15]
def decode(genome):
    sign_bit = genome[0]  # First bit is sign bit
    magnitude = int(genome[1:], 2)

    if sign_bit == '1':  # Negative number
        return -magnitude
    return magnitude

# Fitness function: minimize x^2
def fitness(genome):
    x = decode(genome)
    return x ** 2

# Create random genome (binary string)
def create_genome():
    return ''.join(random.choice('01') for _ in range(GENOME_LENGTH))

# Initialize population
def initialize_population():
    return [create_genome() for _ in range(POPULATION_SIZE)]

# Selection: Tournament selection
def selection(population):
    selected = random.sample(population, TOURNAMENT_SIZE)
    selected.sort(key=fitness)  # Sort by fitness, lower is better
    return selected[0]

# Crossover between two parents to produce two children
def crossover(parent1, parent2):
    if random.random() < CROSSOVER_RATE:
        crossover_point = random.randint(1, GENOME_LENGTH - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]
        return child1, child2
    else:
        return parent1, parent2

# Mutation: flip random bits
def mutate(genome):
    genome = list(genome)
    for i in range(GENOME_LENGTH):
        if random.random() < MUTATION_RATE:
            genome[i] = '1' if genome[i] == '0' else '0'
    return ''.join(genome)

# Main genetic algorithm loop with elitism
def genetic_algorithm():
    # Step 1: Initialize population
    population = initialize_population()

    for generation in range(GENERATIONS):
        new_population = []
        print(f"\nGeneration {generation}:")
        print("-" * 20)
        print(f"Population: {[decode(g) for g in population]}")
        print(f"Population (bits): {population}")

        # Step 2: Elitism - carry over the best individuals unchanged
        population.sort(key=fitness)  # Sort population by fitness
        elites = population[:ELITISM_COUNT]  # Select elites
        new_population.extend(elites)

        print(f"Elites carried forward: {[decode(e) for e in elites]} (bits: {elites})")

        # Step 3: Generate new population (minus elites)
        while len(new_population) < POPULATION_SIZE:
            # Selection
            parent1 = selection(population)
            parent2 = selection(population)

            print(f"Selected Parents: {decode(parent1)} ({parent1}), {decode(parent2)} ({parent2})")

            # Crossover
            child1, child2 = crossover(parent1, parent2)
            print(f"Children before mutation: {decode(child1)} ({child1}), {decode(child2)} ({child2})")

            # Mutation
            child1 = mutate(child1)
            child2 = mutate(child2)
            print(f"Children after mutation: {decode(child1)} ({child1}), {decode(child2)} ({child2})")

            # Add children to new population
            new_population.append(child1)
            new_population.append(child2)

        # Replace old population with new population
        population = new_population[:POPULATION_SIZE]

        # Step 4: Find best solution in current generation
        best_genome = min(population, key=fitness)
        best_fitness = fitness(best_genome)
        print(f"Best solution in this generation: {decode(best_genome)} ({best_genome}), Fitness = {best_fitness}")

        # Early stopping if the best solution is optimal (0 in this case)
        if best_fitness == 0:
            print(f"Optimal solution found at generation {generation}")
            break

    # Final best solution
    best_genome = min(population, key=fitness)
    return decode(best_genome), fitness(best_genome)

In [5]:
# Run the genetic algorithm
best_solution, best_fitness = genetic_algorithm()
print(f"\nBest solution found: x = {best_solution}, Fitness = {best_fitness}")


Generation 0:
--------------------
Population: [76, 105, -2, -3, -68]
Population (bits): ['01001100', '01101001', '10000010', '10000011', '11000100']
Elites carried forward: [-2] (bits: ['10000010'])
Selected Parents: -3 (10000011), -3 (10000011)
Children before mutation: -3 (10000011), -3 (10000011)
Children after mutation: 3 (00000011), -11 (10001011)
Selected Parents: -3 (10000011), -68 (11000100)
Children before mutation: -4 (10000100), -67 (11000011)
Children after mutation: -4 (10000100), -67 (11000011)
Best solution in this generation: -2 (10000010), Fitness = 4

Generation 1:
--------------------
Population: [-2, 3, -11, -4, -67]
Population (bits): ['10000010', '00000011', '10001011', '10000100', '11000011']
Elites carried forward: [-2] (bits: ['10000010'])
Selected Parents: 3 (00000011), -4 (10000100)
Children before mutation: 4 (00000100), -3 (10000011)
Children after mutation: 4 (00000100), -3 (10000011)
Selected Parents: 3 (00000011), -4 (10000100)
Children before mutation